#Importing

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/MyDrive/Sports Analytics/Data Sets')
os.getcwd()
os.listdir()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


['CU_2025_Season.csv']

In [ ]:
import pandas as pd
sports = pd.read_csv('CU_2025_Season.csv', index_col=0)


#Data Visualization

In [ ]:
sports.dtypes
#sports.head()

,0
Offense,object
OffenseScore,int64
Defense,object
DefenseScore,int64
Period,int64
Clock Minutes,int64
Clock Seconds,int64
OffenseTimeouts,int64
DefenseTimeouts,int64
Yardline,int64


Data Cleaning

In [ ]:
sports['Scoring'] = sports['Scoring'].astype(int)
sports = sports[(sports['Offense'] == 'Colorado')]
sports = sports.drop(['OffenseConference','DriveNumber','PlayNumber','OffenseConference','DefenseConference'], axis=1)

In [ ]:
sports = sports[~sports['PlayType'].isin(['Kickoff', 'Fumble Recovery (Opponent)', 'Punt', 'End Period', 'Kickoff Return (Offense)', 'Blocked Field Goal', 'Fumble Recovery (Own)', 'Field Goal Good', 'End of Half','Timeout',
                                          'End of Game','Blocked Punt','Field Goal Missed','Tiemout','Penalty','Safety'])]

In [ ]:
sports['PlayType'].unique()

array(['Pass Reception', 'Rush', 'Passing Touchdown', 'Pass Incompletion',
       'Sack', 'Rushing Touchdown', 'Pass Interception Return'],
      dtype=object)

In [ ]:
#Changing any playtype with 'pass' or 'sack' in it to 'pass'. Same with rush
sports['SimplifiedPlayType'] = sports['PlayType'].apply(lambda x: 'Pass' if 'Pass' in x or x == 'Sack' else ('Rush' if x == 'Rushing Touchdown' else x))
display(sports[['PlayType', 'SimplifiedPlayType']].head())

,PlayType,SimplifiedPlayType
Id,,
401754375101857901,Pass Reception,Pass
401754375101859901,Rush,Rush
401754375101866901,Rush,Rush
401754375101874501,Rush,Rush
401754375101879101,Passing Touchdown,Pass


In [ ]:
sports['SimplifiedPlayType'].unique()

array(['Pass', 'Rush'], dtype=object)

#Logistic Regression

In [ ]:
#Changing Rush and Pass into a binary variable
sports['SimplifiedPlayType'] = sports['SimplifiedPlayType'].map({'Rush': 0, 'Pass': 1})

# Drop rows with NaN in the target variable before defining x and y
sports = sports.dropna(subset=['SimplifiedPlayType'])

#Setting x and y variable
x = sports[['Yardline','YardsToGoal','Down','Distance']]
y = sports['SimplifiedPlayType']

In [ ]:
sports.head()

,Offense,OffenseScore,Defense,DefenseScore,Period,Clock Minutes,Clock Seconds,OffenseTimeouts,DefenseTimeouts,Yardline,YardsToGoal,Down,Distance,YardsGained,Scoring,PlayType,PlayText,Ppa,SimplifiedPlayType
Id,,,,,,,,,,,,,,,,,,,
401754375101857901,Colorado,0,Georgia Tech,0,1,14,20,3,3,64,36,1,10,4,0,Pass Reception,Kaidon Salter pass complete to Quentin Gibson ...,-0.071490,Pass
401754375101859901,Colorado,0,Georgia Tech,0,1,14,0,3,3,68,32,2,6,16,0,Rush,Micah Welch run for 16 yds to the GT 16 for a ...,0.513545,Rush
401754375101866901,Colorado,0,Georgia Tech,0,1,13,30,3,3,84,16,1,10,4,0,Rush,Micah Welch run for 4 yds to the GT 12,-0.020505,Rush
401754375101874501,Colorado,0,Georgia Tech,0,1,12,54,3,3,88,12,2,6,4,0,Rush,Micah Welch run for 4 yds to the GT 8,-0.022637,Rush
401754375101879101,Colorado,7,Georgia Tech,0,1,12,8,3,3,92,8,3,2,8,1,Passing Touchdown,Kaidon Salter pass complete to Dekalon Taylor ...,2.611864,Pass


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
model =LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6666666666666666
Confusion Matrix:
 [[21  9]
 [11 19]]
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.70      0.68        30
           1       0.68      0.63      0.66        30

    accuracy                           0.67        60
   macro avg       0.67      0.67      0.67        60
weighted avg       0.67      0.67      0.67        60



In [ ]:
coefficients = pd.DataFrame({
    'Feature': x.columns,
    'Coefficient': model.coef_[0]
})
print(coefficients)


       Feature  Coefficient
0     Yardline     0.000252
1  YardsToGoal     0.005096
2         Down     0.571000
3     Distance     0.078917
